In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup


In [2]:
#create empty lists
room_type = []
price = []
location = []

#web scraping with selenium
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
driver_path = 'C:\\Users\\celia\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options=options)

# Open the website and perform search
driver.get('https://www.pisocompartido.com/')
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.didomi-components-button.didomi-button.didomi-dismiss-button.didomi-components-button--color.didomi-button-highlight.highlight-button'))).click()
search = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input#inputbusqueda')))
search.clear()
search.send_keys('Madrid')

WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn-buscar.br3.tr05'))).click()
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.listitem.selected'))).click()
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span.btn-continuar.tr05'))).click()

base_url = 'https://www.pisocompartido.com/habitaciones-compartidas-madrid/'
current_page = 1
NUM_PAGES = 51

while current_page <= NUM_PAGES:
    print("Page:", current_page)

    # Wait for the results on the current page to load
    available_rooms = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[8]/div[3]/div[2]')))
    available_rooms_text = available_rooms.get_attribute('innerText')
    available_rooms_lines = available_rooms_text.split('\n')
    print("Rooms found:", len(available_rooms_lines) // 3)
    # Extract the information of the rooms on the current page
    i = 0
    while i < len(available_rooms_lines):
        if 'Habitación' in available_rooms_lines[i]:
            room_type.append(available_rooms_lines[i])
            price.append(available_rooms_lines[i+1])
            location.append(available_rooms_lines[i+2])
            i += 3
        else:
            i += 1

    # Go to the next pages
    next_page_url = base_url + str(current_page + 1)
    driver.get(next_page_url)

    current_page += 1

# Create a DataFrame from the extracted data
data = pd.DataFrame({'Room type': room_type, 'Price': price, 'Location': location})
data.head()

# Save the DataFrame to a CSV file
data.to_csv('piso_compartido.csv', index=False)

C:\Users\celia\AppData\Local\Temp\ipykernel_21420\2639106034.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, options=options)


Page: 1
Rooms found: 33
Page: 2
Rooms found: 33
Page: 3
Rooms found: 28
Page: 4
Rooms found: 28
Page: 5
Rooms found: 28
Page: 6
Rooms found: 29
Page: 7
Rooms found: 29
Page: 8
Rooms found: 29
Page: 9
Rooms found: 29
Page: 10
Rooms found: 29
Page: 11
Rooms found: 28
Page: 12
Rooms found: 29
Page: 13
Rooms found: 29
Page: 14
Rooms found: 28
Page: 15
Rooms found: 28
Page: 16
Rooms found: 29
Page: 17
Rooms found: 27
Page: 18
Rooms found: 28
Page: 19
Rooms found: 28
Page: 20
Rooms found: 28
Page: 21
Rooms found: 28
Page: 22
Rooms found: 28
Page: 23
Rooms found: 28
Page: 24
Rooms found: 28
Page: 25
Rooms found: 29
Page: 26
Rooms found: 28
Page: 27
Rooms found: 29
Page: 28
Rooms found: 28
Page: 29
Rooms found: 29
Page: 30
Rooms found: 28
Page: 31
Rooms found: 28
Page: 32
Rooms found: 29
Page: 33
Rooms found: 28
Page: 34
Rooms found: 29
Page: 35
Rooms found: 28
Page: 36
Rooms found: 28
Page: 37
Rooms found: 28
Page: 38
Rooms found: 29
Page: 39
Rooms found: 29
Page: 40
Rooms found: 29
Page: 41


In [3]:
data.head()

,Room type,Price,Location
0,Habitación Individual,325 €/mes,"Pedrezuela, Madrid"
1,Habitación Doble · Sólo chicos,750 €/mes,"Guindalera, Salamanca, Madrid Capital, Madrid"
2,Habitación Individual · Sólo chicas,320 €/mes,"Casco Histórico, Centro, Alcalá de Henares, Ma..."
3,Habitación Individual,600 €/mes,"Puerta del Ángel, Latina, Madrid Capital, Madrid"
4,Habitación Doble,570 €/mes,"Justicia-Chueca, Centro, Madrid Capital, Madrid"


In [4]:
data.shape

(809, 3)

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
driver_path = 'C:\\Users\\celia\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options=options)

driver.get('https://www.pisocompartido.com/habitaciones-compartidas-madrid/')

# Create an empty list for links
URL_list = []

# Get the links of the ads on the first page
URL_elements = driver.find_elements(By.CSS_SELECTOR, 'div.card')
for URL_element in URL_elements:
    URL = URL_element.find_element(By.TAG_NAME, 'a').get_attribute('href')
    URL_list.append(URL)

# Go to additional pages
current_page = 1
NUM_PAGES = 51

while current_page <= NUM_PAGES:
    next_page_url = f'https://www.pisocompartido.com/habitaciones-compartidas-madrid/{current_page}'
    driver.get(next_page_url)
    
    # Get the links of the ads on the current page
    URL_elements = driver.find_elements(By.CSS_SELECTOR, 'div.card')
    for URL_element in URL_elements:
        URL = URL_element.find_element(By.TAG_NAME, 'a').get_attribute('href')
        URL_list.append(URL)
    
    current_page += 1

# Print the list of room URLs
print("List of room URLs:")
for URL in URL_list:
    print(URL)

driver.quit()


C:\Users\celia\AppData\Local\Temp\ipykernel_21420\2582236937.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, options=options)


List of room URLs:
https://www.pisocompartido.com/habitacion/923683/
https://www.pisocompartido.com/habitacion/928226/
https://www.pisocompartido.com/habitacion/928179/
https://www.pisocompartido.com/habitacion/914213/
https://www.pisocompartido.com/habitacion/928952/
https://www.pisocompartido.com/habitacion/928130/
https://www.pisocompartido.com/habitacion/928948/
https://www.pisocompartido.com/habitacion/928946/
https://www.pisocompartido.com/habitacion/752932/
https://www.pisocompartido.com/habitacion/928945/
https://www.pisocompartido.com/habitacion/917505/
https://www.pisocompartido.com/habitacion/928939/
https://www.pisocompartido.com/habitacion/904670/
https://www.pisocompartido.com/habitacion/881554/
https://www.pisocompartido.com/habitacion/927003/
https://www.pisocompartido.com/habitacion/926998/
https://www.pisocompartido.com/habitacion/923683/
https://www.pisocompartido.com/habitacion/928226/
https://www.pisocompartido.com/habitacion/928179/
https://www.pisocompartido.com/

In [7]:
URL_list

['https://www.pisocompartido.com/habitacion/923683/',
 'https://www.pisocompartido.com/habitacion/928226/',
 'https://www.pisocompartido.com/habitacion/928179/',
 'https://www.pisocompartido.com/habitacion/914213/',
 'https://www.pisocompartido.com/habitacion/928952/',
 'https://www.pisocompartido.com/habitacion/928130/',
 'https://www.pisocompartido.com/habitacion/928948/',
 'https://www.pisocompartido.com/habitacion/928946/',
 'https://www.pisocompartido.com/habitacion/752932/',
 'https://www.pisocompartido.com/habitacion/928945/',
 'https://www.pisocompartido.com/habitacion/917505/',
 'https://www.pisocompartido.com/habitacion/928939/',
 'https://www.pisocompartido.com/habitacion/904670/',
 'https://www.pisocompartido.com/habitacion/881554/',
 'https://www.pisocompartido.com/habitacion/927003/',
 'https://www.pisocompartido.com/habitacion/926998/',
 'https://www.pisocompartido.com/habitacion/923683/',
 'https://www.pisocompartido.com/habitacion/928226/',
 'https://www.pisocompartido

In [10]:
#Remove duplicates
URL_list = list(set(URL_list))


In [13]:
# Add the 'enlaces' list as a new column
data['URL'] = URL

data.to_csv('piso_compartido.csv', index=False)

In [14]:
data

,Room type,Price,Location,URL
0,Habitación Individual,325 €/mes,"Pedrezuela, Madrid",https://www.pisocompartido.com/habitacion/920448/
1,Habitación Doble · Sólo chicos,750 €/mes,"Guindalera, Salamanca, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/
2,Habitación Individual · Sólo chicas,320 €/mes,"Casco Histórico, Centro, Alcalá de Henares, Ma...",https://www.pisocompartido.com/habitacion/920448/
3,Habitación Individual,600 €/mes,"Puerta del Ángel, Latina, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/
4,Habitación Doble,570 €/mes,"Justicia-Chueca, Centro, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/
...,...,...,...,...
804,Habitación Individual,690 €/mes,"Palacio, Centro, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/
805,Habitación Individual,699 €/mes,"Cortes-Huertas, Centro, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/
806,Habitación Individual,690 €/mes,"Palacio, Centro, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/
807,Habitación Individual,550 €/mes,"Palacio, Centro, Madrid Capital, Madrid",https://www.pisocompartido.com/habitacion/920448/


In [15]:
# Create an empty df
df = pd.DataFrame()

for url in URL_list:
    # Perform a GET request to the page
    response = requests.get(url)
    # Create the BeautifulSoup object
    soup = BeautifulSoup(response.content, 'html.parser')

    # Get the desired elements
    elements = soup.find_all('div', class_='contCaract')
    texts = []
    for element in elements:
        spans = element.find_all('span', class_='texto')
        for span in spans:
            text = span.text.strip()
            texts.append(text)

    # Create a dictionary with the characteristics and assign the value 1 to each one
    characteristics = {characteristic: 1 for characteristic in texts}

    # Convert the dictionary into a DataFrame
    df_url = pd.DataFrame(characteristics, index=[0])

    # Add the DataFrame of the link to the main DataFrame
    df = pd.concat([df, df_url], ignore_index=True)

# Fill NaN values with 0
df = df.fillna(0)

df


,Individual,Grande,Amueblada,Llave propia,Hab,Baños,m2,A estrenar,Lavadora,Ascensor,...,JardĆ­n/Terraza,BalcĆ³n,Peque├▒a,BaÃḟos,CalefacciÃġn,JardûÙn/Terraza,Balcû°n propio,PequeĆ±a,BaĆ±os,Jard├¡n/Terraza
0,1.0,1.0,1.0,1.0,1,1.0,1,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,1.0,1,0.0,1,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,1.0,1,0.0,1,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1,0.0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1,1.0,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,0.0,1.0,0.0,1,1.0,1,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
805,1.0,0.0,1.0,1.0,1,0.0,1,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
806,1.0,1.0,1.0,0.0,1,1.0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
807,1.0,1.0,1.0,1.0,1,1.0,1,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df.columns

Index(['Individual', 'Grande', 'Amueblada', 'Llave propia', 'Hab', 'Baños',
       'm2', 'A estrenar', 'Lavadora', 'Ascensor', 'Calefacción', 'Wifi',
       'Mediana', 'Baño', 'Reformado', 'TV', 'Balcón', 'Exterior',
       'En buen estado', 'Plancha', 'PequeĂ±a', 'BaĂ±o', 'CalefacciĂłn',
       'JardĂ­n/Terraza', 'Tendedero', 'Exterior adaptado', 'Secadora',
       'Aire acond.', 'Baño propio', 'Interior adaptado', 'Trastero',
       'Jardín/Terraza', 'Piscina Comunitaria', 'Pequeña',
       'Interior y exterior adaptados', 'Ba├▒o propio', 'Ba├▒os',
       'Calefacci├│n', 'BaÃ±os', 'Doble', 'BaĂ±os', 'CalefacciĂ³n', 'PequeĂąa',
       'BalcĂłn propio', 'BaĂąo', 'BalcĂłn', 'Balcón propio', 'BaĂ±o propio',
       'BalcĂ³n propio', 'Piscina Propia', 'CalefacciÃ³n', 'JardÃ­n/Terraza',
       'BaÃ±o', 'PequeÃ±a', 'PequeÃąa', 'BaÃąo', 'CalefacciÃģn', 'Ba├▒o',
       'Balc├│n', 'BalcĂ³n', 'Baﾃｱos', 'BalcÃ³n', 'PequeûÝa', 'BaûÝo propio',
       'BaûÝos', 'Calefacciû°n', 'Balcû°n', 'Jard├Łn/Te

In [21]:
# Save the df into a csv file
df.to_csv('characteristics_data.csv', index=False)

In [18]:
total_texts = []

for url in URL_list:
    # Perform an HTTP request and get the page content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the desired text
    elements = soup.find_all('div', class_='contTwocols')
    texts = []
    for element in elements:
        col_divs = element.find_all('div', class_='col')
        for col_div in col_divs:
            lis = col_div.find_all('li')
            for li in lis:
                text = li.text.strip()
                texts.append(text)

    # Add the texts to the total result
    total_texts.append(texts)

# Print the total result
for i, texts in enumerate(total_texts):
    print(f"Enlace {i+1}:")
    print(texts)
    print()


Enlace 1:
['Disponible a partir de: 16 de febrero', 'Alquiler mensual: 449 €/Mes', 'Fianza: 449 €', 'Incluye gastos', 'Estancia mínima 2 meses', 'Estancia máxima  Indiferente', 'No se aceptan mascotas', 'No se permiten parejas', 'No se permite fumar', 'Chico o Chica                                                    entre 21 a 35 años', 'No fumador(a)', 'Sin mascotas', 'Sin pareja', '6\r\n                                                                                                                                                                                    Chicos', 'Entre 18 a 99 años']

Enlace 2:
['Disponible a partir de: 1 de junio', 'Alquiler mensual: 380 €/Mes', 'Fianza: 150 €', 'Incluye gastos', 'Estancia mínima +5 meses', 'Estancia máxima +5 meses', 'No se aceptan mascotas', 'No se permiten parejas', 'No se permite fumar', 'Chica                                                    entre 18 a 90 años', 'No fumador(a)', 'Sin mascotas', 'Sin pareja', '2\r\n                  

In [19]:

# Create an empty df
df1 = pd.DataFrame()

# Iterate over the texts from each URL
for texts in total_texts:
    # Create a dictionary with texts as values and indices as columns
    df_enlace1 = pd.DataFrame({text: 1 for text in texts}, index=[0])

    # Fill NaN values with 0
    df_enlace1 = df_enlace1.fillna(0)

    # Add the DataFrame of the URL to the main df
    df1 = pd.concat([df1, df_enlace1], ignore_index=True)

df1 = df1.fillna(0)

df1



,Disponible a partir de: 16 de febrero,Alquiler mensual: 449 €/Mes,Fianza: 449 €,Incluye gastos,Estancia mínima 2 meses,Estancia máxima Indiferente,No se aceptan mascotas,No se permiten parejas,No se permite fumar,Chico o Chica entre 21 a 35 años,...,Alquiler mensual: 330 ā¬/Mes,Fianza: 100 ā¬,Chica entre 18 a 40 aĆ±os,Entre 18 a 40 aĆ±os,Chico o Chica entre 24 a 26 aÃąos,Entre 24 a 26 aÃąos,Fianza: 1.248 €,Chico o Chica entre 20 a 34 años,Entre 20 a 28 años,Disponible a partir de: 20 de agosto
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
806,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
807,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df1.to_csv('characteristics_data2.csv', index=False)